# Carregando Variáveis de Ambiente do Arquivo .env

Usamos python-dotenv para carregar as variáveis de ambiente do arquivo .env.


In [22]:
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, text
from IPython.display import display
import pandas as pd
import time
from math import radians, cos, sin, asin, sqrt

# Carregar variáveis do arquivo .env
load_dotenv()

True

# Configuração do Ambiente Spark

Configuramos a sessão do Spark para otimizar o processamento de grandes volumes de dados.


In [2]:
# Recriar a sessão Spark usando as configurações locais definidas em spark-defaults.conf
spark = SparkSession.builder \
    .appName("HiveTableCreation") \
    .enableHiveSupport() \
    .getOrCreate()

# Configurações adicionais do Hive
spark.conf.set("hive.metastore.warehouse.dir", "file:///C:/spark/hive/warehouse")
spark.conf.set("hive.metastore.uri", "thrift://localhost:9083")


# Verificando o Spark

In [3]:
# Verificação do Spark
try:
    # Exibir a versão do Spark para verificar se está funcional
    print(f"Versão do Spark: {spark.version}")
    print("Spark está levantado e funcional.")
except Exception as e:
    print("Erro ao levantar o Spark:", e)


Versão do Spark: 3.5.0
Spark está levantado e funcional.


# Verificando o Hadoop [Usando o Power Shell]

Para que o Hadoop funcione normalmente em ambiente de aprendizagem, onde a máquina local é desligada e depois ligada para novas operações, será necessário no Power Shell como administrador (win + x) executar os comandos a seguir a partir da pasta `C:\hadoop\sbin`:

1. **Verificar e remover processos usando portas específicas:**

    ```powershell
    netstat -ano | findstr :9000
    taskkill /PID <PID> /F

    netstat -ano | findstr :9866
    taskkill /PID <PID> /F
    ```

2. **Obter o nome de usuário e conceder permissões:**

    ```powershell
    whoami /user  # Pegue no resultado desse comando seu nome de usuário (até a barra "\")
    ```

    ```powershell
    icacls "C:\hadoop\data\namenode" /grant "<user>:F" /T
    icacls "C:\hadoop\data\datanode" /grant "<user>:F" /T
    ```

3. **Remover dados antigos das pastas do DataNode e NameNode:**

    ```powershell
    Remove-Item -Path "C:\hadoop\data\datanode\*" -Recurse -Force
    Remove-Item -Path "C:\hadoop\data\namenode\*" -Recurse -Force
    ```

4. **Formatar o NameNode e iniciar serviços do Hadoop:**

    ```powershell
    hdfs namenode -status  # Aguarde até realizar todos os processos. Responda Y se solicitado.
    .\start-dfs.cmd  # Aguarde até a conclusão dos processos.
    .\start-yarn.cmd  # Ativar o processo YARN.
    ```

5. **Acessar as URLs do Hadoop para verificar:**

    - HDFS: [http://localhost:9870/](http://localhost:9870/)
    - YARN: [http://localhost:8088/](http://localhost:8088/)

### Comandos HDFS

1. **Criar uma pasta no cluster:**

    ```powershell
    hdfs dfs -mkdir -p /nome_pasta
    ```

2. **Remover um arquivo específico no cluster:**

    ```powershell
    hdfs dfs -rm -r /dados/arquivo.parquet
    ```

3. **Setar permissões para uma pasta ou arquivo:**

    ```powershell
    hdfs dfs -chmod -R 755 /dados/arquivo.parquet
    ```

4. **Exibir metadados do arquivo no caminho especificado:**

    ```powershell
    hdfs dfs -ls /dados/arquivo.parquet
    ```

5. **Exibir os arquivos na raiz do cluster:**

    ```powershell
    hdfs dfs -ls /
    ```


# Salvar o CSV no formato PARQUET no HDFS

Lemos os dados do arquivo `temas_ambientais.csv`, e salvamos no HDFS.



In [5]:

hdfs_parquet_path = None

try:
    print("Iniciando leitura do arquivo CSV e conversão para Parquet no HDFS")
    
    # Caminho local do arquivo CSV
    csv_path = "data/temas_ambientais.csv"
    
    # Caminho no HDFS para salvar o Parquet
    hdfs_parquet_path = "hdfs://localhost:9000/dados/temas_ambientais.parquet"
    
    # Leitura do CSV com o separador ';'
    df = spark.read.csv(csv_path, header=True, inferSchema=True, sep=';')
    
    # Salvamento em Parquet no HDFS
    df.write.mode("overwrite").parquet(hdfs_parquet_path)
    
    print(f"Arquivo salvo com sucesso no HDFS: {hdfs_parquet_path}")
    print(f"Total de registros: {df.count()}")
    print("Schema do DataFrame:")
    df.printSchema()

except Exception as e:
    print(f"Erro no processamento: {e}")


Iniciando leitura do arquivo CSV e conversão para Parquet no HDFS
Arquivo salvo com sucesso no HDFS: hdfs://localhost:9000/dados/temas_ambientais.parquet
Total de registros: 6839104
Schema do DataFrame:
root
 |-- uf: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- codigo_ibge: integer (nullable = true)
 |-- area_do_imovel: double (nullable = true)
 |-- registro_car: string (nullable = true)
 |-- situacao_cadastro: string (nullable = true)
 |-- condicao_cadastro: string (nullable = true)
 |-- area_liquida: double (nullable = true)
 |-- area_remanescente_vegetacao_nativa: double (nullable = true)
 |-- area_reserva_legal_proposta: double (nullable = true)
 |-- area_preservacao_permanente: double (nullable = true)
 |-- area_nao_classificada: double (nullable = true)
 |-- solicitacao_adesao_pra: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- data_inscricao: timestamp (nullable = true)
 |-- data_alteracao_

# Modificar o nome das Colunas

In [6]:
# Função para formatar nomes de colunas (remover underscores e capitalizar)
def format_column_name(col_name):
    return ' '.join(word.capitalize() for word in col_name.split('_'))

# Ler o DataFrame Parquet
df = spark.read.parquet(hdfs_parquet_path)

# Renomear colunas
for old_col in df.columns:
    new_col = format_column_name(old_col)
    df = df.withColumnRenamed(old_col, new_col)

# Criar tabela temporária

In [8]:
# Nome da tabela Hive
table_name = "temas_ambientais"

# Dropar tabela existente
spark.sql(f"DROP TABLE IF EXISTS {table_name}")

# Criar view temporária
df.createOrReplaceTempView("temp_table")

# Comando SQL para criar tabela Hive
create_table_sql = f"""
  CREATE TABLE IF NOT EXISTS {table_name}
  STORED AS PARQUET
  LOCATION '{hdfs_parquet_path}'
  AS SELECT * FROM temp_table
"""

# Executar criação da tabela
spark.sql(create_table_sql)


DataFrame[]

#  Consulta SQL usando Hive com medição de tempo

Exibimos informações úteis sobre o tempo de consultas SQL usando o Hive no Dataframe.



In [ ]:
# Configurar exibição
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

# Início do temporizador para consulta
tempo_inicio_consulta = time.time()

# Executar a consulta completa sem limitar o DataFrame
resultado = spark.sql(f"SELECT * FROM {table_name}")
numero_de_linhas = resultado.count()
tempo_consulta = time.time() - tempo_inicio_consulta

# Exibir os resultados da consulta
print(f"Tempo de consulta SQL: {tempo_consulta:.2f} segundos")
print(f"Número de linhas no DataFrame: {numero_de_linhas}")

# Usar display para mostrar o DataFrame no Jupyter Notebook
display(resultado.limit(5).toPandas())

# Exibir o schema da tabela
print("\nEstrutura da Tabela:")
display(spark.sql(f"DESCRIBE {table_name}"))


Tempo de consulta SQL: 0.63 segundos
Número de linhas no DataFrame: 13678208


,Uf,Municipio,Codigo Ibge,Area Do Imovel,Registro Car,Situacao Cadastro,Condicao Cadastro,Area Liquida,Area Remanescente Vegetacao Nativa,Area Reserva Legal Proposta,...,Data Alteracao Condicao Cadastro,Area Rural Consolidada,Area Servidao Administrativa,Tipo Imovel Rural,Modulos Fiscais,Area Uso Restrito,Area Reserva Legal Averbada,Area Reserva Legal Aprovada Nao Averbada,Area Pousio,Data Ultima Retificacao
0,PA,Melgaço,1504505,8.6572,PA-1504505-60C7BD9561D442D3A35BC962553E3593,AT,"Aguardando análise, não passível de revisão de...",8.6572,8.657153,0.0000,...,2022-05-06 04:24:49.156,0.000000,0.0,IRU,0.1237,0.0,0.0,0.0,0.0,2022-05-06 04:24:33.206
1,PA,Melgaço,1504505,8.9118,PA-1504505-38CEB3B058ED45DFB94267ADB879921F,AT,"Aguardando análise, não passível de revisão de...",8.9118,8.911800,0.0000,...,2022-05-06 04:25:25.227,0.000000,0.0,IRU,0.1273,0.0,0.0,0.0,0.0,2022-05-06 04:25:08.862
2,PA,Vitória do Xingu,1508357,541.3827,PA-1508357-F2AD1C32877C492689615CCEF1F0D7CA,PE,"Analisado com pendências, aguardando retificaç...",541.3827,0.000000,89.4643,...,2022-05-23 23:03:40.040,450.127674,0.0,IRU,7.2184,0.0,0.0,0.0,0.0,2022-05-06 04:25:44.570
3,CE,Cascavel,2303501,0.0845,CE-2303501-9A25D7359BBC4C21A57856A2DFA6391D,AT,"Aguardando análise, não passível de revisão de...",0.0845,0.000000,0.0000,...,2022-05-06 04:30:34.965,0.083406,0.0,IRU,0.0030,0.0,0.0,0.0,0.0,2022-05-06 04:30:18.634
4,SP,Borborema,3507407,64.2994,SP-3507407-2DAD18BEF2AA46CE9A8F550332055488,AT,"Revisado, aguardando análise da equipe",64.3080,0.794852,0.7949,...,2022-06-09 11:08:09.043,0.000000,0.0,IRU,4.0192,0.0,0.0,0.0,0.0,2022-05-06 05:00:55.199



Estrutura da Tabela:


col_name,data_type,comment
Uf,string,NULL
Municipio,string,NULL
Codigo Ibge,int,NULL
Area Do Imovel,double,NULL
Registro Car,string,NULL
Situacao Cadastro,string,NULL
Condicao Cadastro,string,NULL
Area Liquida,double,NULL
Area Remanescente...,double,NULL
Area Reserva Lega...,double,NULL


# Primeira Consulta SQL usando Hive do Projeto

Recupere a soma de área que é a coluna "Area Do Imovel" para todas as propriedades agrícolas que pertecem ao MS e MT que estão na coluna "Uf". Ordene o resultado em ordem descrescente.

***Obs: Crie a pasta para a tabela Hive. Use o comando "hdfs dfs -mkdir -p /tmp/hive" no Power Shell como Administrador***.



In [12]:
try:
    # Início do temporizador total
    tempo_inicio_total = time.time()   

    # Consulta SQL: Soma da Área para MS e MT
    consulta_1 = """
    SELECT Uf, SUM(`Area Do Imovel`) AS Soma_Area
    FROM temp_table
    WHERE Uf IN ('MS', 'MT')
    GROUP BY Uf
    ORDER BY Soma_Area DESC
    """

    # Salvar os resultados da consulta no HDFS
    df_consulta_1 = spark.sql(consulta_1)  # Primeira consulta do projeto
    table_consulta_1 = "projeto_consulta_1"
    df_consulta_1.write.mode("overwrite").parquet(f"hdfs://localhost:9000/tmp/hive/{table_consulta_1}")
    print(f"Tabela {table_consulta_1} criada no HDFS com os resultados da consulta 1.")
    
    # Ler a tabela criada no HDFS e exibir os resultados completos
    consulta_1 = spark.read.parquet(f"hdfs://localhost:9000/tmp/hive/{table_consulta_1}")
    numero_de_linhas = consulta_1.count()
    display(consulta_1.limit(5).toPandas())    
    print(f"Número de linhas no DataFrame: {numero_de_linhas}") 

    # Tempo total de execução
    tempo_total = time.time() - tempo_inicio_total
    print(f"\nTempo total de execução: {tempo_total:.2f} segundos")

except Exception as e:
    print(f"Erro no processamento: {e}")


Tabela projeto_consulta_1 criada no HDFS com os resultados da consulta 1.


,Uf,Soma_Area
0,MT,8.458709e+07
1,MS,3.632188e+07


Número de linhas no DataFrame: 2

Tempo total de execução: 3.63 segundos


# Segunda Consulta SQL usando Hive do Projeto

filtrar todas as propriedades que pertencem à região sudeste do Brasil.

In [13]:
try:
    # Início do temporizador total
    tempo_inicio_total = time.time()   

    # Consulta SQL: Filtrar propriedades da região sudeste
    consulta_2 = """
    SELECT *
    FROM temp_table
    WHERE Uf IN ('SP', 'RJ', 'MG', 'ES')
    """

    # Salvar os resultados da consulta no HDFS
    df_consulta_2 = spark.sql(consulta_2)  # Segunda consulta do projeto
    table_consulta_2 = "projeto_consulta_2"
    df_consulta_2.write.mode("overwrite").parquet(f"hdfs://localhost:9000/tmp/hive/{table_consulta_2}")
    print(f"Tabela {table_consulta_2} criada no HDFS com os resultados da consulta 2.")
    
    # Ler a tabela criada no HDFS e exibir os resultados completos
    consulta_2 = spark.read.parquet(f"hdfs://localhost:9000/tmp/hive/{table_consulta_2}")
    numero_de_linhas = consulta_2.count()
    display(consulta_2.limit(5).toPandas())    
    print(f"Número de linhas no DataFrame: {numero_de_linhas}") 

    # Tempo total de execução
    tempo_total = time.time() - tempo_inicio_total
    print(f"\nTempo total de execução: {tempo_total:.2f} segundos")

except Exception as e:
    print(f"Erro no processamento: {e}")


Tabela projeto_consulta_2 criada no HDFS com os resultados da consulta 2.


,Uf,Municipio,Codigo Ibge,Area Do Imovel,Registro Car,Situacao Cadastro,Condicao Cadastro,Area Liquida,Area Remanescente Vegetacao Nativa,Area Reserva Legal Proposta,...,Data Alteracao Condicao Cadastro,Area Rural Consolidada,Area Servidao Administrativa,Tipo Imovel Rural,Modulos Fiscais,Area Uso Restrito,Area Reserva Legal Averbada,Area Reserva Legal Aprovada Nao Averbada,Area Pousio,Data Ultima Retificacao
0,SP,Brodowski,3507803,7.2293,SP-3507803-EF2877CFB21F4443BF3B7F8665ED3164,AT,"Revisado, aguardando aceite pelo proprietário",7.2368,0.000000,0.0000,...,2022-02-03 13:15:22.810,0.000000,0.000000,IRU,0.0000,0.0,0.0,0.0,0.0,2015-05-09 22:53:32.800
1,SP,Dracena,3514403,22.2074,SP-3514403-51883E2CA225418DA0153F9CEE900C90,AT,"Revisado, aguardando aceite pelo proprietário",22.1915,0.000000,0.0000,...,2022-02-04 07:53:04.730,0.751033,0.000000,IRU,1.1104,0.0,0.0,0.0,0.0,2015-05-09 22:53:36.283
2,MG,Pimenta,3150505,8.2017,MG-3150505-F68352DCF0B840C68D04D752DC71FADF,AT,"Aguardando análise, não passível de revisão de...",8.2017,0.394558,0.3946,...,NaT,7.796744,0.000000,IRU,0.2300,0.0,0.0,0.0,0.0,2015-05-09 22:53:37.507
3,SP,Paraguaçu Paulista,3535507,19.8508,SP-3535507-4AE4E04B91AF484185E20160AD1A3ADE,AT,"Revisado, aguardando aceite pelo proprietário",19.6332,0.000000,0.0000,...,2022-09-10 11:29:42.531,0.000000,0.202827,IRU,0.9824,0.0,0.0,0.0,0.0,2015-05-09 22:53:40.686
4,SP,Laranjal Paulista,3526407,6.7043,SP-3526407-0C2A559800844D26BDC7D34D0D74DD22,AT,"Revisado, aguardando aceite pelo proprietário",6.6269,0.451086,0.0000,...,2022-09-09 22:44:35.084,0.084980,0.086547,IRU,0.3677,0.0,0.0,0.0,0.0,2015-05-09 22:53:44.385


Número de linhas no DataFrame: 1565746

Tempo total de execução: 15.11 segundos


# Terceira Consulta SQL usando Hive do Projeto

calcular quantas propriedades foram cadastradas por ano, utilizando a coluna data_inscricao e agrupando por ano. Aqui já fazemos uma transformação exibindo somente as colunas que não são vazias.

In [18]:
try:
    # Início do temporizador total
    tempo_inicio_total = time.time()   

    # Consulta SQL: Contar propriedades cadastradas por ano, filtrando nulos
    consulta_3 = """
    SELECT YEAR(`Data Inscricao`) AS Ano, COUNT(*) AS Total_Propriedades
    FROM temp_table
    WHERE `Data Inscricao` IS NOT NULL
    GROUP BY YEAR(`Data Inscricao`)
    ORDER BY Ano
    """

    # Salvar os resultados da consulta no HDFS
    df_consulta_3 = spark.sql(consulta_3)  # Terceira consulta do projeto
    table_consulta_3 = "projeto_consulta_3"
    df_consulta_3.write.mode("overwrite").parquet(f"hdfs://localhost:9000/tmp/hive/{table_consulta_3}")
    print(f"Tabela {table_consulta_3} criada no HDFS com os resultados da consulta 3.")
    
    # Ler a tabela criada no HDFS e exibir os resultados completos
    consulta_3 = spark.read.parquet(f"hdfs://localhost:9000/tmp/hive/{table_consulta_3}")
    numero_de_linhas = consulta_3.count()
    display(consulta_3.toPandas())    
    print(f"Número de linhas no DataFrame: {numero_de_linhas}") 

    # Tempo total de execução
    tempo_total = time.time() - tempo_inicio_total
    print(f"\nTempo total de execução: {tempo_total:.2f} segundos")

except Exception as e:
    print(f"Erro no processamento: {e}")


Tabela projeto_consulta_3 criada no HDFS com os resultados da consulta 3.


,Ano,Total_Propriedades
0,2013,743
1,2014,227468
2,2015,1240178
3,2016,1828786
4,2017,794779
5,2018,724856
6,2019,746570
7,2020,463322
8,2021,443942
9,2022,368448


Número de linhas no DataFrame: 10

Tempo total de execução: 1.61 segundos


# Quarta Consulta SQL usando Hive do Projeto

calcular o percentual médio de área remanescente de vegetação nativa em relação à área total do proprietário.

In [19]:
try:
    # Início do temporizador total
    tempo_inicio_total = time.time()   

    # Consulta SQL: Percentual médio de área remanescente de vegetação nativa
    consulta_4 = """
    SELECT 
        AVG((`Area Remanescente Vegetacao Nativa` / `Area Do Imovel`) * 100) AS Percentual_Medio
    FROM temp_table
    """

    # Salvar os resultados da consulta no HDFS
    df_consulta_4 = spark.sql(consulta_4)  # Quarta consulta do projeto
    table_consulta_4 = "projeto_consulta_4"
    df_consulta_4.write.mode("overwrite").parquet(f"hdfs://localhost:9000/tmp/hive/{table_consulta_4}")
    print(f"Tabela {table_consulta_4} criada no HDFS com os resultados da consulta 4.")
    
    # Ler a tabela criada no HDFS e exibir os resultados completos
    consulta_4 = spark.read.parquet(f"hdfs://localhost:9000/tmp/hive/{table_consulta_4}")
    numero_de_linhas = consulta_4.count()
    display(consulta_4.toPandas())    
    print(f"Número de linhas no DataFrame: {numero_de_linhas}") 

    # Tempo total de execução
    tempo_total = time.time() - tempo_inicio_total
    print(f"\nTempo total de execução: {tempo_total:.2f} segundos")

except Exception as e:
    print(f"Erro no processamento: {e}")

Tabela projeto_consulta_4 criada no HDFS com os resultados da consulta 4.


,Percentual_Medio
0,18.087406


Número de linhas no DataFrame: 1

Tempo total de execução: 1.28 segundos


# Quinta Consulta SQL usando Hive do Projeto

contar as propriedades rurais por estado. 

In [21]:
try:
    # Início do temporizador total
    tempo_inicio_total = time.time()   

    # Consulta SQL: Contagem de propriedades rurais por estado
    consulta_5 = """
    SELECT Uf, COUNT(*) AS Total_Propriedades_Rurais
    FROM temp_table    
    GROUP BY Uf
    ORDER BY Uf
    """

    # Salvar os resultados da consulta no HDFS
    df_consulta_5 = spark.sql(consulta_5)  # Quinta consulta do projeto
    table_consulta_5 = "projeto_consulta_5"
    df_consulta_5.write.mode("overwrite").parquet(f"hdfs://localhost:9000/tmp/hive/{table_consulta_5}")
    print(f"Tabela {table_consulta_5} criada no HDFS com os resultados da consulta 5.")
    
    # Ler a tabela criada no HDFS e exibir os resultados completos
    consulta_5 = spark.read.parquet(f"hdfs://localhost:9000/tmp/hive/{table_consulta_5}")
    numero_de_linhas = consulta_5.count()
    display(consulta_5.toPandas())    
    print(f"Número de linhas no DataFrame: {numero_de_linhas}") 

    # Tempo total de execução
    tempo_total = time.time() - tempo_inicio_total
    print(f"\nTempo total de execução: {tempo_total:.2f} segundos")

except Exception as e:
    print(f"Erro no processamento: {e}")


Tabela projeto_consulta_5 criada no HDFS com os resultados da consulta 5.


,Uf,Total_Propriedades_Rurais
0,AC,45141
1,AL,114610
2,AM,73642
3,AP,10116
4,BA,1012499
5,CE,314866
6,DF,17339
7,ES,108849
8,GO,199690
9,MA,273642


Número de linhas no DataFrame: 27

Tempo total de execução: 1.16 segundos


# Sexta Consulta SQL usando Hive do Projeto

calcular a distância entre a maior propriedade e Brasília utilizando as coordenadas de centróide da propriedade.

**Usaremos a fórmula de Haversine para calcular a distância entre dois pontos geográficos.**

In [25]:
def calcular_distancia(lat1, lon1, lat2, lon2):
    """
    Calcula a distância entre dois pontos geográficos usando a fórmula de Haversine.
    """
    # Converter de graus para radianos
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Fórmula de Haversine
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Raio da Terra em quilômetros
    return c * r

try:
    # Início do temporizador total
    tempo_inicio_total = time.time()   

    # Consulta SQL: Selecionar a maior propriedade
    consulta_6 = """
    SELECT `Area Do Imovel`, `Latitude`, `Longitude`, `Uf`, `Municipio`
    FROM temp_table
    ORDER BY `Area Do Imovel` DESC
    LIMIT 1
    """

    # Executar a consulta para obter a maior propriedade
    df_consulta_6 = spark.sql(consulta_6)
    resultado = df_consulta_6.collect()[0]  # Obter o resultado da consulta

    # Extrair as coordenadas da maior propriedade e outras informações
    latitude_propriedade = resultado["Latitude"]
    longitude_propriedade = resultado["Longitude"]
    area_propriedade = resultado["Area Do Imovel"]
    uf_propriedade = resultado["Uf"]
    municipio_propriedade = resultado["Municipio"]

    # Coordenadas de Brasília
    latitude_brasilia = -15.796943053171708
    longitude_brasilia = -47.891638482569476

    # Calcular a distância usando a fórmula de Haversine
    distancia = calcular_distancia(latitude_propriedade, longitude_propriedade, latitude_brasilia, longitude_brasilia)
    
    # Criar um DataFrame com os resultados
    data = [(area_propriedade, latitude_propriedade, longitude_propriedade, distancia, uf_propriedade, municipio_propriedade)]
    columns = ["Area Do Imovel", "Latitude", "Longitude", "Distancia_Brasilia", "Uf", "Municipio"]
    df_resultado = spark.createDataFrame(data, columns)

    # Salvar os resultados da consulta no HDFS
    table_consulta_6 = "projeto_consulta_6"
    df_resultado.write.mode("overwrite").parquet(f"hdfs://localhost:9000/tmp/hive/{table_consulta_6}")
    print(f"Tabela {table_consulta_6} criada no HDFS com os resultados da consulta 6.")
    
    # Ler a tabela criada no HDFS e exibir os resultados completos
    consulta_6 = spark.read.parquet(f"hdfs://localhost:9000/tmp/hive/{table_consulta_6}")
    numero_de_linhas = consulta_6.count()
    display(consulta_6.toPandas())    
    print(f"Número de linhas no DataFrame: {numero_de_linhas}") 

    # Tempo total de execução
    tempo_total = time.time() - tempo_inicio_total
    print(f"\nTempo total de execução: {tempo_total:.2f} segundos")

except Exception as e:
    print(f"Erro no processamento: {e}")


Tabela projeto_consulta_6 criada no HDFS com os resultados da consulta 6.


,Area Do Imovel,Latitude,Longitude,Distancia_Brasilia,Uf,Municipio
0,2.420079e+06,-5.475619,-68.888188,2562.129615,AM,Jutaí


Número de linhas no DataFrame: 1

Tempo total de execução: 19.79 segundos


# Sétima Consulta SQL usando Hive do Projeto

calcular a média de todas as propriedades e, em seguida, determinar quantas propriedades por estado estão acima da média.

**Vamos usar GROUP BY para agrupar os estados e contar as propriedades acima da média.**

In [26]:
try:
    # Início do temporizador total
    tempo_inicio_total = time.time()   

    # Consulta SQL: Calcular a média de área de todas as propriedades
    consulta_media = """
    SELECT AVG(`Area Do Imovel`) AS Media_Area
    FROM temp_table
    """
    df_media = spark.sql(consulta_media)
    media_area = df_media.collect()[0]["Media_Area"]

    # Consulta SQL: Contagem de propriedades por estado acima da média
    consulta_7 = f"""
    SELECT Uf, COUNT(*) AS Total_Propriedades_Acima_Media
    FROM temp_table
    WHERE `Area Do Imovel` > {media_area}
    GROUP BY Uf
    ORDER BY Uf
    """

    # Salvar os resultados da consulta no HDFS
    df_consulta_7 = spark.sql(consulta_7)  # Sétima consulta do projeto
    table_consulta_7 = "projeto_consulta_7"
    df_consulta_7.write.mode("overwrite").parquet(f"hdfs://localhost:9000/tmp/hive/{table_consulta_7}")
    print(f"Tabela {table_consulta_7} criada no HDFS com os resultados da consulta 7.")
    
    # Ler a tabela criada no HDFS e exibir os resultados completos
    consulta_7 = spark.read.parquet(f"hdfs://localhost:9000/tmp/hive/{table_consulta_7}")
    numero_de_linhas = consulta_7.count()
    display(consulta_7.toPandas())    
    print(f"Número de linhas no DataFrame: {numero_de_linhas}") 

    # Tempo total de execução
    tempo_total = time.time() - tempo_inicio_total
    print(f"\nTempo total de execução: {tempo_total:.2f} segundos")

except Exception as e:
    print(f"Erro no processamento: {e}")


Tabela projeto_consulta_7 criada no HDFS com os resultados da consulta 7.


,Uf,Total_Propriedades_Acima_Media
0,AC,10559
1,AL,3641
2,AM,21412
3,AP,3759
4,BA,50508
5,CE,22931
6,DF,1040
7,ES,6378
8,GO,61385
9,MA,42202


Número de linhas no DataFrame: 27

Tempo total de execução: 1.59 segundos
